
<center>
        <h1>CS 251/340 Machine Learning</h1>
        <h2>AUA, Spring 2021</h2>
        <h1>Here We introduce step by step implementatiion of the Pseudocode presented on the Google Docs By Aram Khanlari<h1>

</center>

**Importing neccessery libraries**


In [ ]:
from music21 import *

#uploading file from computer
import os
#Array Processing
import numpy as np
import random

!pip install mingus
import mingus.core.notes as notes

from collections import Counter

#library for visualiation
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

#for Neural Networks
from keras.layers import *
from keras.models import *
from keras.callbacks import *
import keras.backend as K

from keras.models import load_model

# **Function to read MIDI files**

In [ ]:

chordes = []


#Returns the array of notes and chords present in the musical file.
def read_midi(file):
    
    print("Loading the music file: ",file)
    
    notes=[]
    notes_to_parse = None
    #parsing a midi file
    midi = converter.parse(file)
  
    #grouping based on different instruments
    s2 = instrument.partitionByInstrument(midi)
    #Looping over all the instruments
    for part in s2.parts:
    
        #select elements of only piano
        #if 'Piano' in str(part): 
        
            notes_to_parse = part.recurse() 
      
            #finding whether a particular element is note or a chord
            for element in notes_to_parse:
                
                #note
                if isinstance(element, note.Note):
                    b = str(element.pitch)
                    notes.append(str(element.pitch))
                    #print(b)
                
                #chord
                elif isinstance(element, chord.Chord):
                    a = '.'.join(str(n) for n in element.pitchClasses)
                    #notes.append(a)
                   # print(a)
                    chordes.append(a)
                    #a =  notes.append('.'.join(str(element.pitch)))
                    #print(a)

    return  np.array(notes)
#np.array(chordes)

In [ ]:
#specify the path
path='/bin/a/'


#read all the filenames
files=[i for i in os.listdir(path) if i.endswith(".mid")]

#reading each midi file
notes_array = np.array([read_midi(path+i) for i in files])
print(len(notes_array[0]))

Loading the music file:  /bin/a/Chopin_Prelude_in_E_Minor_Opus_28_No_4.mid
76


# **Function to convert list of notes/chordes back to MIDI**

In [ ]:
def convert_to_midi(prediction_output):
   
    offset = 0
    output_notes = []

    # create note and chord objects based on the values generated by the model
    for pattern in prediction_output:
        
        # pattern is a chord
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                
                cn=int(current_note)
                new_note = note.Note(cn)
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
                
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
            
        # pattern is a note
        else:
            
            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)

        # increase offset each iteration so that notes do not stack
        offset += 1
    midi_stream = stream.Stream(output_notes)
    midi_stream.write('midi', fp='predicted_music.mid')
    return midi_stream
    

# **Reading MIDI files from the uploaded folder**

In [ ]:

#specify the path
path='/bin/m/'


#read all the filenames
files=[i for i in os.listdir(path) if i.endswith(".mid")]

#reading each midi file
notes_array = np.array([read_midi(path+i)[1] for i in files])


Loading the music file:  /bin/m/schub_d760_1.mid
Loading the music file:  /bin/m/schu_143_3.mid
Loading the music file:  /bin/m/schu_143_2.mid
Loading the music file:  /bin/m/schub_d760_3.mid
Loading the music file:  /bin/m/schu_143_1.mid
Loading the music file:  /bin/m/schub_d760_4.mid
Loading the music file:  /bin/m/schub_d760_2.mid
Loading the music file:  /bin/m/schub_d960_1.mid


In [ ]:
def remove_double_single_chordes(chordes):
  new_list = chordes.copy()
  for i in new_list:
    if i.count('.') == 1:
      chordes.remove(i)
    elif len(i) <= 2:
      chordes.remove(i)
  return chordes



Coverting 2D Notes/Chordes array into 1D

In [ ]:
notes_ = [element for note_ in notes_array for element in note_]
print(notes_)

['E', '3', 'B', '4', 'F', '3', 'G', '3', 'A', '3', 'D', '3', 'E', '-', '4', 'B', '-', '2']


# **Layer 1: Random**
# The melody A = {x1, x2, …, xn} is harmonized with random chords C = {c1, c2, …, cn} where chords are chosen with no logic or distribution.

Function to Randomly select chordes from the list and generate music out of it

In [ ]:
def random_chords_to_music(notes_, melody):
    
  new_music=[]
  chordes = []
  
  for i in range(len(melody[0])): 
    j = random.randint(0, len(notes_))
    new_chord = notes_[j] + "." + str(notes.note_to_int(str(melody[0][i])))
    new_music.append(new_chord)
   

  convert_to_midi(new_music)
    

In [ ]:
a =  remove_double_single_chordes(chordes)
random_chords_to_music(a, notes_)
print(a)

['7.11.4', '7.11.4', '7.11.4', '7.11.4', '7.11.4', '7.11.4', '7.11.4', '7.11.4', '6.9.4', '6.9.4', '6.9.4', '6.9.4', '6.9.3', '6.9.3', '6.9.3', '6.9.3', '5.9.3', '5.9.3', '5.9.3', '5.9.3', '5.9.2', '5.9.2', '5.8.2', '5.8.2', '4.8.2', '4.8.2', '4.8.2', '4.8.2', '4.7.2', '4.7.2', '4.7.1', '4.7.1', '4.7.0', '4.7.0', '4.7.0', '4.7.0', '4.6.0', '4.6.0', '4.6.0', '4.6.0', '4.6.0', '4.6.0', '4.6.0', '4.6.0', '3.6.0', '3.6.0', '3.6.0', '3.6.0', '2.6.0', '2.6.0', '2.6.0', '2.6.0', '2.6.0', '2.6.0', '2.6.0', '2.6.0', '2.5.0', '2.5.0', '2.5.0', '2.5.0', '2.5.11', '2.5.11', '2.5.11', '2.5.11', '0.4.11', '0.4.11', '0.4.9', '0.4.9', '0.4.9', '0.4.9', '0.4.9', '0.4.9', '11.4.9', '11.4.9', '11.3.9', '11.3.9', '0.4.9', '0.4.9', '0.4.9', '0.4.9', '11.3.9', '11.3.9', '11.3.9', '11.3.9', '0.4.9', '0.4.9', '0.4.9', '0.4.9', '11.3.9', '7.11.4', '7.11.4', '7.11.4', '7.11.4', '7.11.4', '7.11.4', '7.11.4', '7.11.4', '6.9.4', '6.9.4', '6.9.4', '6.9.4', '5.9.3', '5.9.3', '5.9.3', '5.9.3', '5.8.3', '5.8.3', '5.8.

#**Layer 2a: Pseudorandom**
#The melody A = {x1, x2, …, xn} is harmonized with chords C = {c1, c2, …, cn} where a chord ci is chosen such that it contains the note xi. The decision for the chords is still random, however, the decision pool is narrowed down from all chords to those which contain the note


In [ ]:
def pseudorandom_chords_to_music(notes_to_choose, note):
  new_music=[]
  for i in range(len(notes_to_choose)): 
    i = random.randint(0, len(notes_to_choose)-1)
    if note in notes_to_choose[i]:
      new_music.append(notes_to_choose[i])
  
  convert_to_midi(new_music)
    

In [ ]:
pseudorandom_chords_to_music(chordes,"10")

# **Layer 2b: Pseudorandom Smooth**
The melody A = {x1, x2, …, xn} is harmonized with chords C = {c1, c2, …, cn} where a chord ci is chosen such that it contains the note xi. The decision pool is narrowed down even further to those chords which minimize the function 

F(ci-1, ci) = min(|ci-11-ci1|, 12-|ci-11-ci1|) + min(|ci-12-ci2|, 12-|ci-12-ci2|) + min(|ci-13-ci3|,12-|ci-13-ci3|),  

where cij is the j-th note of the three-note i chord. In simple words, this function measures and adds all the distances between notes of two chords. The smaller the distance is, the smoother, “lazier” the harmony would move. For example: F([do, mi, sol],[fa, la, do]) = min(|do-fa|, 12-|do-fa|) + min(|mi-la|, 12-|mi-la|) + min(|sol-do|,12-|sol-do|). Do - fa is the distance between the notes in chromatic (halfstep, կիսատոն) counts. For example for do - fa it is -5 (do, do#, re, re#, mi, fa) conversely, fa - do is 5. Thus F([do, mi, sol],[fa, la, do]) =min(5,7) + min(5,7) + min(5,7) = 15. Another example is F([do, mi, sol],[do, fa, la]) = min(|do-do|, 12-|do-do|) + min(|mi-fa|, 12-|mi-fa|) + min(|sol-la|,12-|sol-la|) = min(0,12) + min(1,11) + min(2,10) = 3.
